In [1]:
import requests
import numpy as np
import tensorflow as tf
import msgpack
import json
import zlib
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.int64)
x_train = x_train.reshape(-1, 28*28)


In [2]:

r = requests.get('http://localhost:3030/')

print(r)
print(r.text)

<Response [200]>
{"scale_base":1.3,"leaf_cutoff":10,"min_res_index":-20,"use_singletons":true,"partition_type":"First","verbosity":0,"rng_seed":0}


In [3]:
sample_bytes = zlib.compress(msgpack.packb([float(f) for f in x_train[0]]))

r = requests.get('http://localhost:3030/knn', params = {"k": 5}, headers = {"Content-Type": "gzip"}, data=sample_bytes)

print(r)
print(json.dumps(json.loads(r.text), indent=2))

<Response [200]>
{
  "knn": [
    {
      "name": "0",
      "distance": 0.0
    },
    {
      "name": "32248",
      "distance": 6.123421
    },
    {
      "name": "8728",
      "distance": 6.241574
    },
    {
      "name": "18932",
      "distance": 6.2538
    },
    {
      "name": "30483",
      "distance": 6.2616057
    }
  ]
}


In [4]:
sample_bytes = zlib.compress(msgpack.packb([float(f) for f in x_train[0]]))

r = requests.get('http://localhost:3030/routing_knn', params = {"k": 5}, headers = {"Content-Type": "gzip"}, data=sample_bytes)

print(r)
print(json.dumps(json.loads(r.text), indent=2))

<Response [200]>
{
  "routing_knn": [
    {
      "name": "629",
      "distance": 8.008285
    },
    {
      "name": "15210",
      "distance": 8.067046
    },
    {
      "name": "35390",
      "distance": 8.094733
    },
    {
      "name": "24038",
      "distance": 8.235575
    },
    {
      "name": "38281",
      "distance": 8.591897
    }
  ]
}


In [5]:
sample_bytes = zlib.compress(msgpack.packb([float(f) for f in x_train[0]]))

r = requests.get('http://localhost:3030/path', params = {"k": 5}, headers = {"Content-Type": "gzip"}, data=sample_bytes)

print(r)
print(json.dumps(json.loads(r.text), indent=2))

<Response [200]>
{
  "path": [
    {
      "name": "59999",
      "layer": 11,
      "distance": 9.924044
    },
    {
      "name": "59999",
      "layer": 10,
      "distance": 9.924044
    },
    {
      "name": "59999",
      "layer": 9,
      "distance": 9.924044
    },
    {
      "name": "15210",
      "layer": 8,
      "distance": 8.067046
    }
  ]
}


In [6]:
# Should 200, {'TrackPath': {'success': False}}
sample_bytes = zlib.compress(msgpack.packb([float(f) for f in x_train[0]]))

r = requests.post('http://localhost:3030/track/point', headers = {"Content-Type": "gzip"}, data=sample_bytes)

print(r)
print(json.dumps(json.loads(r.text), indent=2))

<Response [200]>
{
  "TrackPath": {
    "success": false
  }
}


In [7]:
# Should 404, no window
r = requests.get('http://localhost:3030/track/stats')

print(r)

<Response [404]>


In [8]:
# Should get {'Unknown': [None, 100]}
r = requests.get('http://localhost:3030/track/stats?window_size=100')

print(r)
print(json.dumps(json.loads(r.text), indent=2))

<Response [200]>
{
  "Unknown": [
    null,
    100
  ]
}


In [9]:
# Should get {'Unknown': [None, 100]}
r = requests.post('http://localhost:3030/track/add?window_size=100')

print(r)
print(json.dumps(json.loads(r.text), indent=2))

<Response [200]>
{
  "AddTracker": {
    "success": true
  }
}


In [10]:
# Should 200, {'TrackPath': {'success': True}}
sample_bytes = zlib.compress(msgpack.packb([float(f) for f in x_train[0]]))

r = requests.post('http://localhost:3030/track/point', headers = {"Content-Type": "gzip"}, data=sample_bytes)

print(r)
print(json.dumps(json.loads(r.text), indent=2))

<Response [200]>
{
  "TrackPath": {
    "success": true
  }
}


In [11]:
# Should get a current stats object with a very small kl_div.
r = requests.get('http://localhost:3030/track/stats?window_size=100')

print(r)
print(json.loads(r.text))

<Response [200]>
{'CurrentStats': {'kl_div': 0.05452124172559536, 'max': 0.056572243432235325, 'min': 2.6088748938946082e-09, 'nz_count': 3, 'moment1_nz': 0.05657476608674905, 'moment2_nz': 0.00320041873330673, 'sequence_len': 1}}


A user queries the data they are interested in. Their distribution is not like the training data, but it should be relatively close.

In [12]:
# Should get a current stats object with a very small kl_div.
r = requests.post('http://localhost:3030/track/add?window_size=100&tracker_name=normal')

for i in range(100):
    sample_bytes = zlib.compress(msgpack.packb([float(f) for f in x_train[i]]))
    r = requests.post('http://localhost:3030/track/point?tracker_name=normal', headers = {"Content-Type": "gzip"}, data=sample_bytes)
    assert json.loads(r.text)["TrackPath"]["success"] == True
    
r = requests.get('http://localhost:3030/track/stats?window_size=100&tracker_name=normal')

print(r)
print(json.dumps(json.loads(r.text), indent=2))

<Response [200]>
{
  "CurrentStats": {
    "kl_div": 10.54270919064686,
    "max": 0.5435121254599649,
    "min": 2.6368420392941516e-05,
    "nz_count": 127,
    "moment1_nz": 11.914924413607055,
    "moment2_nz": 2.3793493272263535,
    "sequence_len": 100
  }
}


An attacker queries the same spot (near abouts) many times over. Their distribution is not like the training data, it's a dirac delta

In [13]:
# Should get a current stats object with a very small kl_div.
r = requests.post('http://localhost:3030/track/add?window_size=100&tracker_name=attack')

sample_bytes = zlib.compress(msgpack.packb([float(f) for f in x_train[0]]))
for i in range(100):
    r = requests.post('http://localhost:3030/track/point?tracker_name=attack', headers = {"Content-Type": "gzip"}, data=sample_bytes)
    assert json.loads(r.text)["TrackPath"]["success"] == True
    
r = requests.get('http://localhost:3030/track/stats?window_size=100&tracker_name=attack')

print(r)
print(json.dumps(json.loads(r.text), indent=2))

<Response [200]>
{
  "CurrentStats": {
    "kl_div": 78.28138439428824,
    "max": 176.15371611160458,
    "min": 2.6368420392941516e-05,
    "nz_count": 3,
    "moment1_nz": 176.178909811383,
    "moment2_nz": 31030.132333323043,
    "sequence_len": 100
  }
}
